In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score

# 可視化して内容10点確認
for i,(x,y) in enumerate(zip(train_X[:10],train_y[:10])):
  tXimg=np.reshape(x,(8,8))
  plt.subplot(2,5,i+1)
  plt.title("training:%i"%y)
  plt.imshow(tXimg, cmap=plt.cm.gray_r, interpolation='nearest')

data = load_digits()
train_X, test_X, train_y, test_y = train_test_split(
    data.data, data.target, random_state=42)

max_score = 0
best_param = None
estimators = ["K近傍法","決定木","ランダムフォレスト","サポートベクタマシン(SVM)","xgboost"]

# estimator使い分け
for estimator in estimators:
  if estimator == "K近傍法":
    print(f"{estimator}探索中")
    model_param_set_grid = { KNeighborsClassifier() : {"n_neighbors": [i for i in range(3,6)],
                                                       "weights":["uniform","distance"],
                                                       "algorithm":["ball_tree","kd_tree","brute"],
                                                       "leaf_size":[i for i in range(20,50,10)]}}
  elif estimator == "決定木":
    print(f"{estimator}探索中")
    model_param_set_grid = { DecisionTreeClassifier() : {"criterion": ["gini","entropy"],
                                                         "max_depth": [5,10,15],
                                                         "random_state":[42]}}

  elif estimator == "ランダムフォレスト":
    print(f"{estimator}探索中")
    model_param_set_grid = { RandomForestClassifier() : {"n_estimators": [i for i in range(80,140,20)],
                                                         "max_depth":[10,15, 20],
                                                         "random_state":[42],
                                                         "class_weight":["balanced",None]}}

  elif estimator == "サポートベクタマシン(SVM)":
    print(f"{estimator}探索中")
    model_param_set_grid = { SVC() : {"C": [1, 10, 100],
                                      "kernel":["linear","rbf","sigmoid","poly"],
                                      "gamma":["scale",0.1,1.0,10],
                                      "class_weight":["balanced",None],
                                      "random_state":[42]}}

  else:
    print(f"{estimator}探索中")
    model_param_set_grid = { XGBClassifier() : {"max_depth": [10,15,20],
                                                "subsample":[0.5,0.8,1],
                                                "colsample_bytree":[0.5,0.8,1],
                                                "learning_rate":[0.01,0.3,0.5],
                                                "n_jobs":[-1]}}


  # グリッドサーチでパラメーターサーチ
  for model, param in model_param_set_grid.items():
    clf = GridSearchCV(model,param)
    clf.fit(train_X, train_y)
    pred_y = clf.predict(test_X)
    # モデルのF値を計算
    score = f1_score(test_y,pred_y,average="micro")
    print(f"学習モデル：{estimator}")
    print(f"パラメーター：{clf.best_params_}")
    print(f"ベストスコア：{score}")
    if max_score < score:
      max_score = score
      best_param = clf.best_params_
      best_model = estimator

print("\n")
print("最適学習モデル:{},\n最適パラメーター:{}".format(best_model, best_param))
# 最も成績のいいスコアを出力
print("最終ベストスコア:{}".format(max_score))